# STGNN WAVE
Bunu implement ettim
https://github.com/dmlc/dgl/tree/master/examples/pytorch/stgcn_wave

Buna da bakmalıyım bi ara
https://github.com/chnsh/DCRNN_PyTorch

In [1]:
import dgl
import torch
import numpy as np
import pandas as pd
import scipy.sparse as sp

from sklearn.preprocessing import StandardScaler

pd.options.display.float_format = "{:,.2f}".format

Using backend: pytorch[16:57:34] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/mert/miniconda3/envs/thesis_torch/lib/python3.9/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.2.so: cannot open shared object file: No such file or directory



In [2]:
def data_transform(data, n_window, n_ahead, device=torch.device("cpu")):
    """
    :param data: rows for time indices, columns for locations
        currently does not support features
    :param n_window: history length
    :param n_ahead: next time index to predict
    """
    # produce data slices for training and testing
    n_obs, n_loc = data.shape
    
    num = n_obs - n_window - n_ahead
    x = np.zeros([num, 1, n_window, n_loc]) # features might end up being in second dimension
    y = np.zeros([num, n_loc])
    
    count = 0
    for start in range(num):
        x[count, :, :, :] = data[start: start + n_window].reshape(1, n_window, n_loc)
        y[count] = data[start + n_window + n_ahead - 1]
        count += 1
    return torch.Tensor(x).to(device), torch.Tensor(y).to(device)

In [3]:
data = pd.read_parquet("data/wind/2019-01-24_outlier_removed.parquet")    
data = data[~data["rt_plant_id"].isin([2397, 2420, 2538])]
assert data.rt_plant_id.nunique() == 98

In [4]:
plant_mapping = {k:v for k,v in zip(np.sort(data.rt_plant_id.astype(int).unique()), range(98))}
data.rt_plant_id = data.rt_plant_id.map(plant_mapping) # zaten sort ettiği için gerek yok aslında
rev_plant_mapping = {v:k for k,v in plant_mapping.items()}
rev_plant_mapping.get(0)


672

In [5]:
data = data[(data["forecast_dt"] > "2020-10-31") & (data["forecast_dt"] < "2021-01-01")]
data.head()

,epoch,production,rt_plant_id,forecast_dt,UGRD_80.m.above.ground.SW,UGRD_80.m.above.ground.NW,UGRD_80.m.above.ground.NE,UGRD_80.m.above.ground.SE,VGRD_80.m.above.ground.SW,VGRD_80.m.above.ground.NW,VGRD_80.m.above.ground.NE,VGRD_80.m.above.ground.SE,production_lower,production_upper,is_outlier,production_cleaned
15499,1604095200,7.00,24,2020-10-31 01:00:00+03:00,-5.70,-4.12,-4.04,-3.13,-5.22,-6.16,-3.91,-2.28,0.00,10.00,False,7.00
15500,1604098800,6.00,24,2020-10-31 02:00:00+03:00,-4.58,-3.58,-3.88,-2.38,-4.12,-5.72,-2.72,-1.02,0.00,10.00,False,6.00
15501,1604102400,2.00,24,2020-10-31 03:00:00+03:00,-3.79,-2.79,-2.79,-1.69,-3.49,-5.09,-1.99,-0.89,0.00,10.00,False,2.00
15502,1604106000,2.00,24,2020-10-31 04:00:00+03:00,-4.57,-4.07,-3.47,-2.27,-2.85,-3.95,-1.75,-0.95,0.00,10.00,False,2.00
15503,1604109600,1.00,24,2020-10-31 05:00:00+03:00,-4.76,-4.46,-3.56,-3.16,-2.58,-2.78,-1.78,-1.58,0.00,10.00,False,1.00


In [8]:
print(data.forecast_dt.nunique(), data.forecast_dt.nunique() / 24)

1487 61.958333333333336


In [9]:
def generate_adjacency_matrix(df, threshold=0.5):    
    data = df[["forecast_dt", "rt_plant_id", "production"]]
    data = pd.pivot_table(data, index="forecast_dt", columns="rt_plant_id", values="production")

    A = abs(data.corr())
    A = A[A > threshold].fillna(0)
    return A

In [ ]:
G = nx.from_numpy_matrix(A.values)

In [102]:
adj = generate_adjacency_matrix(data)
sp_mx = sp.coo_matrix(adj)
G = dgl.from_scipy(sp_mx, eweight_name="weight")

print(len(G.all_edges()[0]))
print(len(G.edata["weight"]))

G = dgl.remove_self_loop(G)
print(len(G.all_edges()[0]))
print(len(G.edata["weight"]))

G = dgl.add_self_loop(G)
print(len(G.all_edges()[0]))
print(len(G.edata["weight"]))

6584
6584
6486
6486
6584
6584


In [113]:
G.ndata

{}

In [11]:
data.head()

,epoch,production,rt_plant_id,forecast_dt,UGRD_80.m.above.ground.SW,UGRD_80.m.above.ground.NW,UGRD_80.m.above.ground.NE,UGRD_80.m.above.ground.SE,VGRD_80.m.above.ground.SW,VGRD_80.m.above.ground.NW,VGRD_80.m.above.ground.NE,VGRD_80.m.above.ground.SE,production_lower,production_upper,is_outlier,production_cleaned
15499,1604095200,7.00,24,2020-10-31 01:00:00+03:00,-5.70,-4.12,-4.04,-3.13,-5.22,-6.16,-3.91,-2.28,0.00,10.00,False,7.00
15500,1604098800,6.00,24,2020-10-31 02:00:00+03:00,-4.58,-3.58,-3.88,-2.38,-4.12,-5.72,-2.72,-1.02,0.00,10.00,False,6.00
15501,1604102400,2.00,24,2020-10-31 03:00:00+03:00,-3.79,-2.79,-2.79,-1.69,-3.49,-5.09,-1.99,-0.89,0.00,10.00,False,2.00
15502,1604106000,2.00,24,2020-10-31 04:00:00+03:00,-4.57,-4.07,-3.47,-2.27,-2.85,-3.95,-1.75,-0.95,0.00,10.00,False,2.00
15503,1604109600,1.00,24,2020-10-31 05:00:00+03:00,-4.76,-4.46,-3.56,-3.16,-2.58,-2.78,-1.78,-1.58,0.00,10.00,False,1.00


In [12]:
weather_cols = [col for col in data.columns if col.startswith(("VGRD", "UGRD"))]
data_pivot = pd.pivot_table(
    data[["rt_plant_id", "forecast_dt", "production", *weather_cols]], 
    index="forecast_dt", 
    columns="rt_plant_id", 
    )
data_pivot.head()

UGRD_80.m.above.ground.NE                         \
rt_plant_id                                      0     1    2     3     4    
forecast_dt                                                                  
2020-10-31 01:00:00+03:00                      4.05  1.95 1.37  0.45 -0.66   
2020-10-31 02:00:00+03:00                      4.22  0.42 3.52 -1.98 -0.08   
2020-10-31 03:00:00+03:00                      3.11 -1.19 2.71 -3.59 -0.39   
2020-10-31 04:00:00+03:00                      1.83 -2.67 0.83 -2.57 -0.57   
2020-10-31 05:00:00+03:00                      2.94 -3.46 0.24 -2.96 -0.36   

                                                      ... production       \
rt_plant_id                  5    6    7    8     9   ...         88   89   
forecast_dt                                           ...                   
2020-10-31 01:00:00+03:00 -4.04 1.41 2.57 1.35  0.19  ...      20.00 0.20   
2020-10-31 02:00:00+03:00 -3.88 1.42 3.62 1.32 -0.08  ...      21.00 1.10   
2020-10-31 03:00:00+03:00 -2.79 1.21 3.81 1.01 -0.29  ...      26.00 3.26   
2020-10-31 04:00:00+03:00 -3.47 0.93 2.93 0.53 -0.27  ...      18.00 5.14   
2020-10-31 05:00:00+03:00 -3.56 0.34 1.84 0.04  0.14  ...      17.00 8.42   

                                                                      
rt_plant_id                  90    91   92   93   94   95    96   97  
forecast_dt                                                           
2020-10-31 01:00:00+03:00  8.47  6.73 2.25 3.40 0.55 1.66 12.00 0.80  
2020-10-31 02:00:00+03:00  8.40  4.65 3.32 2.84 1.33 4.06  9.00 0.40  
2020-10-31 03:00:00+03:00  5.59  7.68 3.10 2.70 1.53 5.07 10.00 1.11  
2020-10-31 04:00:00+03:00  8.49 10.21 2.98 2.70 1.10 5.81  4.00 1.10  
2020-10-31 05:00:00+03:00 12.07  8.15 4.93 2.11 1.38 7.31  8.00 4.00  

[5 rows x 882 columns]

In [13]:
def data_transform(data, n_window, n_ahead, device=torch.device("cpu")):
    """
    :param data: rows for time indices, columns for locations
        currently does not support features
    :param n_window: history length
    :param n_ahead: next time index to predict
    """
    # produce data slices for training and testing
    n_obs, n_loc = data["production"].shape
    num = n_obs - n_window - n_ahead + 1
    weather_cols = [col for col in data.columns.get_level_values(0).unique() if col != "production"]
    n_features = len(weather_cols)

    x = np.zeros([num, 1, n_window, n_loc]) # features might end up being in second dimension
    y = np.zeros([num, n_loc])
    f = np.zeros([num, n_features, n_window, n_loc])

    count = 0
    for start in range(num):
        x[count, :, :, :] = data["production"][start: start + n_window].values.reshape(1, n_window, n_loc)
        y[count] = data["production"].values[start + n_window + n_ahead - 1]
        count += 1

    for i in range(n_features):
        count = 0
        for start in range(num):
            f[count, i, :, :] = data[weather_cols[i]][start: start + n_window].values.reshape(1, n_window, n_loc)
            count += 1

    return torch.Tensor(x).to(device), torch.Tensor(y).to(device), torch.Tensor(f).to(device)

In [68]:
scaler = StandardScaler()
scaler.fit_transform(data_pivot.values)

array([[ 2.23041085,  1.46646812,  1.01582766, ..., -1.18471524,
        -0.41548285, -1.2310813 ],
       [ 2.28455863,  0.93781888,  1.90309461, ..., -0.944506  ,
        -0.65964949, -1.28453717],
       [ 1.92633782,  0.3833234 ,  1.56944347, ..., -0.84341794,
        -0.57826061, -1.18965299],
       ...,
       [ 1.02849939,  1.20570154,  0.62496801, ...,  0.96715922,
         1.04951701, -0.60297474],
       [ 0.96644017,  1.22575281,  0.33910114, ...,  0.90610604,
         1.04951701,  0.19886341],
       [ 1.14976182,  1.17635073,  0.54849252, ...,  1.09326907,
         1.04951701,  0.45946081]])

In [22]:
# 50 -> 2
# 50 - 29 + 1 -> 2
# 1487 - (48 + 1) + 1 = 1439

In [15]:
n_window = 48
x, y, f = data_transform(data_pivot, n_window=n_window, n_ahead=1)

In [17]:
x.shape, y.shape, f.shape

(torch.Size([1439, 1, 48, 98]),
 torch.Size([1439, 98]),
 torch.Size([1439, 8, 48, 98]))

In [61]:
# first
i = 0 
print(np.isclose(x[i].reshape(-1, 98).cpu().numpy(), data_pivot["production"].iloc[i:i+n_window].values).all())
print(np.isclose(y[i].reshape(-1, 98).cpu().numpy(), data_pivot["production"].iloc[i+n_window].values).all())

# last
i = data_pivot.shape[0] - 48 - 1 
print(np.isclose(x[i].reshape(-1, 98).cpu().numpy(), data_pivot["production"].iloc[i:i+n_window].values).all())
print(np.isclose(y[i].reshape(-1, 98).cpu().numpy(), data_pivot["production"].iloc[i+n_window].values).all())

True
True
True
True


(torch.Size([1439, 1, 48, 98]),
 torch.Size([1439, 98]),
 torch.Size([1439, 8, 48, 98]))

In [79]:
# scaler = StandardScaler()
# scaler.fit_transform(x)

In [88]:
print(x.shape, y.shape, f.shape)

train_ratio = 0.9
val_ratio = 0.1

len_val = round(x.shape[0] * val_ratio)
len_train = round(x.shape[0] * train_ratio)

x_train = x[:len_train]
y_train = y[:len_train]
f_train = f[:len_train]

x_val = x[len_train:len_train + len_val]
y_val = y[len_train:len_train + len_val]
f_val = f[len_train:len_train + len_val]

print(x_train.shape, y_train.shape, f_train.shape)
print(x_val.shape, y_val.shape, f_val.shape)


torch.Size([1439, 1, 48, 98]) torch.Size([1439, 98]) torch.Size([1439, 8, 48, 98])
torch.Size([1295, 1, 48, 98]) torch.Size([1295, 98]) torch.Size([1295, 8, 48, 98])
torch.Size([144, 1, 48, 98]) torch.Size([144, 98]) torch.Size([144, 8, 48, 98])


In [ ]:
# train_ratio = 0.9
# val_ratio = 0.099
# len_val = round(num_samples * val_ratio)
# len_train = round(num_samples * train_ratio)

# train_indices = [_ for _ in range(len_train)]
# val_indices = [len_train + _ for _ in range(len_val)]
# test_indices = [_ for _ in range(len(df)) if _ not in train_indices + val_indices]

# train = df[train_indices]
# val = df[val_indices]
# test = df[test_indices]

# print(len(train), len(val), len(test))

# scaler = StandardScaler()
# train = scaler.fit_transform(train)
# val = scaler.transform(val)
# test = scaler.transform(test)

In [ ]:
# df[val_indices]

# df_.iloc[val_indices]

# df_.iloc[train_indices + val_indices]

# pd.DataFrame(scaler.inverse_transform(train))

# plt.plot(np.append(train[:, 0], val[:, 0]))

# plt.plot(df[train_indices + val_indices, 0])

In [ ]:
# plant_id = 0
# df_.iloc[train_indices + val_indices, plant_id].plot(figsize=(20,10))
# # draw vertical line at the end of training period
# plt.axvline(x=df_.index[len_train], color="red")
# plt.title(rev_plant_mapping.get(plant_id))


In [89]:
# n_window, n_ahead = 2 * 24, 1
# x_train, y_train = data_transform(train, n_window, n_ahead)
# x_val, y_val = data_transform(val, n_window, n_ahead)
# # x_test, y_test = data_transform(test, n_window, n_ahead)

In [90]:
batch_size = 50
train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_iter = torch.utils.data.DataLoader(train_data, batch_size)
val_data = torch.utils.data.TensorDataset(x_val, y_val)
val_iter = torch.utils.data.DataLoader(val_data, batch_size)
# test_data = torch.utils.data.TensorDataset(x_test, y_test)
# test_iter = torch.utils.data.DataLoader(test_data, batch_size)

In [91]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv

class TemporalConvLayer(nn.Module):
    ''' Temporal convolution layer.
    
    arguments
    ---------
    c_in : int
        The number of input channels (features)
    c_out : int
        The number of output channels (features)
    dia : int
        The dilation size
    '''
    def __init__(self, c_in, c_out, dia = 1):
        super(TemporalConvLayer, self).__init__()
        self.c_out = c_out
        self.c_in = c_in
        self.conv = nn.Conv2d(c_in, c_out, (2, 1), 1, dilation = dia, padding = (0,0))


    def forward(self, x):
        return torch.relu(self.conv(x))


class SpatioConvLayer(nn.Module):
    def __init__(self, c, Lk): # c : hidden dimension Lk: graph matrix
        super(SpatioConvLayer, self).__init__()
        self.g = Lk
        self.gc = GraphConv(c, c, activation=F.relu)

    def init(self):
        stdv = 1. / math.sqrt(self.W.weight.size(1))
        self.W.weight.data.uniform_(-stdv, stdv)

    def forward(self, x):
        x = x.transpose(0, 3)
        x = x.transpose(1, 3)
        output = self.gc(self.g, x)
        output = output.transpose(1, 3)
        output = output.transpose(0, 3)
        return torch.relu(output)

class FullyConvLayer(nn.Module):
    def __init__(self, c):
        super(FullyConvLayer, self).__init__()
        self.conv = nn.Conv2d(c, 1, 1)

    def forward(self, x):
        return self.conv(x)

class OutputLayer(nn.Module):
    def __init__(self, c, T, n):
        super(OutputLayer, self).__init__()
        self.tconv1 = nn.Conv2d(c, c, (T, 1), 1, dilation = 1, padding = (0,0))
        self.ln = nn.LayerNorm([n, c])
        self.tconv2 = nn.Conv2d(c, c, (1, 1), 1, dilation = 1, padding = (0,0))
        self.fc = FullyConvLayer(c)

    def forward(self, x):
        x_t1 = self.tconv1(x)
        x_ln = self.ln(x_t1.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
        x_t2 = self.tconv2(x_ln)
        return self.fc(x_t2)

class STGCN_WAVE(nn.Module):
    def __init__(self, c, T, n, Lk, device=torch.device("cpu"), control_str='TNTSTNTST'):
        super(STGCN_WAVE, self).__init__()
        self.control_str = control_str
        self.num_layers = len(control_str)
        self.layers = nn.ModuleList([])
        cnt = 0
        diapower = 0
        for i in range(self.num_layers):
            i_layer = control_str[i]
            if i_layer == 'T': # Temporal Layer
                self.layers.append(TemporalConvLayer(c[cnt], c[cnt + 1], dia = 2**diapower))
                diapower += 1
                cnt += 1
            if i_layer == 'S': # Spatio Layer
                self.layers.append(SpatioConvLayer(c[cnt], Lk))
            if i_layer == 'N': # Norm Layer
                self.layers.append(nn.LayerNorm([n,c[cnt]]))
        self.output = OutputLayer(c[cnt], T + 1 - 2**(diapower), n)
        for layer in self.layers:
            layer = layer.to(device)
    
    def forward(self, x):
        for i in range(self.num_layers):
            i_layer = self.control_str[i]
            if i_layer == 'N':
                x = self.layers[i](x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)  
            else:
                x = self.layers[i](x)
        return self.output(x)

In [94]:
blocks = [1, 16, 32, 64, 32, 128]
lr = 0.001
num_nodes = 98

loss = nn.MSELoss()
device=torch.device("cpu")
G = G.to(device)
model = STGCN_WAVE(blocks, n_window, num_nodes, G).to(device)

optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

In [95]:
save_path = "stgcnwavemodel.pt"
epochs = 5

def evaluate_model(model, loss, data_iter):
    model.eval()
    l_sum, n = 0.0, 0
    with torch.no_grad():
        for x, y in data_iter:
            y_pred = model(x).view(len(x), -1)
            l = loss(y_pred, y)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        return l_sum / n


def evaluate_metric(model, data_iter, scaler):
    model.eval()
    with torch.no_grad():
        mae, mape, mse = [], [], []
        for x, y in data_iter:
            y = scaler.inverse_transform(y.cpu().numpy()).reshape(-1)
            y_pred = scaler.inverse_transform(model(x).view(len(x), -1).cpu().numpy()).reshape(-1)
            d = np.abs(y - y_pred)
            mae += d.tolist()
            mape += (d / y).tolist()
            mse += (d ** 2).tolist()
        MAE = np.array(mae).mean()
        MAPE = np.array(mape).mean()
        RMSE = np.sqrt(np.array(mse).mean())
        return MAE, MAPE, RMSE

In [96]:
from time import time
min_val_loss = np.inf

In [97]:
for epoch in range(1, epochs + 1):
    l_sum, n = 0.0, 0
    model.train()
    t_ = time()
    for x, y in train_iter:
        y_pred = model(x).view(len(x), -1)
        l = loss(y_pred, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        l_sum += l.item() * y.shape[0]
        n += y.shape[0]
    scheduler.step()
    val_loss = evaluate_model(model, loss, val_iter)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), save_path)
    print("epoch", epoch, ", time:", round(time() - t_, 2),  ", train loss:", l_sum / n, ", validation loss:", val_loss)

    

/home/mert/miniconda3/envs/thesis_torch/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


epoch 1 , time: 25.78 , train loss: 713.2379415504721 , validation loss: 877.7399953206381
epoch 2 , time: 24.75 , train loss: 674.7976600912086 , validation loss: 849.1674321492513
epoch 3 , time: 24.99 , train loss: 638.8578657349104 , validation loss: 784.6833953857422
epoch 4 , time: 24.01 , train loss: 607.1410323919937 , validation loss: 748.0721638997396
epoch 5 , time: 24.31 , train loss: 579.8067423993556 , validation loss: 656.9690772162544


In [98]:
# adj matrixi train datadan oluştur

In [99]:
best_model = STGCN_WAVE(blocks, n_window, num_nodes, G).to(device)
best_model.load_state_dict(torch.load(save_path))

<All keys matched successfully>

In [100]:
l = evaluate_model(best_model, loss, val_iter)
MAE, MAPE, RMSE = evaluate_metric(best_model, val_iter, scaler)
print("test loss:", l, "\nMAE:", MAE, ", MAPE:", MAPE, ", RMSE:", RMSE)

ValueError: operands could not be broadcast together with shapes (50,98) (882,) (50,98) 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot(y, y_pred, i=0):
    y_pred = scaler.inverse_transform(y_pred.cpu().numpy())
    y = scaler.inverse_transform(y.cpu().numpy())
    plt.plot(y[:, i], label="y")
    plt.plot(y_pred[:, i], label="y_pred")
    plt.legend()
    plt.show()

In [ ]:
def plot_pred(model, data_iter, plant_id=0):
    model.eval()
    with torch.no_grad():
        x, y = list(data_iter)[0]
        y_pred = model(x).view(len(x), -1)
    y_pred = scaler.inverse_transform(y_pred.cpu().numpy())
    y = scaler.inverse_transform(y.cpu().numpy())
    plt.plot(y[:, plant_id], label="y")
    plt.plot(y_pred[:, plant_id], label="y_pred")
    plt.legend()
    plt.show()

In [ ]:
for i, t in enumerate(train_iter):
    print(i, t[0].shape, t[1].shape)

for i, t in enumerate(val_iter):
    print(i, t[0].shape, t[1].shape)

In [ ]:
plot_pred(best_model, train_iter, plant_id=4)

In [ ]:
plot_pred(best_model, val_iter, plant_id=8)

In [ ]:
model = best_model
data_iter = val_iter


In [ ]:
y_list, y_pred_list, x_list = [], [], []
model.eval()
with torch.no_grad():
    for data in data_iter:
        x, y = data
        y_pred = model(x).view(len(x), -1)
        y_list.append(y.cpu().numpy())
        x_list.append(x.cpu().numpy())
        y_pred_list.append(y_pred.cpu().numpy())

y_list = np.concatenate(y_list)
x_list = np.concatenate(x_list)
y_pred_list = np.concatenate(y_pred_list)

y_list = scaler.inverse_transform(y_list)
# x_list = scaler.inverse_transform(x_list)
y_pred_list = scaler.inverse_transform(y_pred_list)

In [ ]:
for x_ in x_list:
    x_df_ = pd.DataFrame(scaler.inverse_transform(x_[0]))
    print(x_df_.iloc[-3:, :3])

In [ ]:
pd.DataFrame(scaler.inverse_transform(x_list[1][0])).head().append(pd.DataFrame(scaler.inverse_transform(x_list[1][0])).tail(5))

In [ ]:
pd.DataFrame(scaler.inverse_transform(x_list[0][0])).head().append(pd.DataFrame(scaler.inverse_transform(x_list[0][0])).tail(5))

In [ ]:
pd.DataFrame(y_list[0].reshape(1, -1))

In [ ]:
pd.DataFrame(y_pred_list[0].reshape(1, -1))

In [ ]:
data_iter.sampler

In [ ]:
[i for i in dir(data_iter) if "__" not in i]

In [ ]:
len(list(data_iter)[0])

In [ ]:
plant_id = 4
# figsize
# fig, ax = plt.subplots(figsize=(20, 10))

y_values = y_list[:, plant_id]
y_pred_values = y_pred_list[:, plant_id]

# ax.plot(y_values, label="y")
# ax.plot(y_pred_values, label="y_pred")
# fig.show()

In [ ]:
output = df_.iloc[train_indices + val_indices, plant_id]


In [ ]:
output = pd.DataFrame(df_.iloc[train_indices + val_indices, plant_id])
output.columns = ["production"]
output

In [ ]:
output["y"] = np.nan
output.iloc[48:48+len(y_values), 1] = y_values
output.iloc[48:]

In [ ]:
len(y_values)

In [ ]:
np.relen(y_values)

In [ ]:
797 - 48 - 1

In [ ]:
len(y_values), len(train_indices), len(val_indices)

In [ ]:
plant_id = 4
df_.iloc[train_indices + val_indices, plant_id].plot(figsize=(20,10))
# draw vertical line at the end of training period
plt.axvline(x=df_.index[len_train], color="red")
plt.title(rev_plant_mapping.get(plant_id))


In [ ]:
print(x_val.shape, y_val.shape)

In [ ]:
def to_df(x, transform=True):
    x = x.reshape(-1, 98).cpu().numpy()
    if transform:
        x = scaler.inverse_transform(x)
    return pd.DataFrame(x)

In [ ]:
df_[round(0.5*len(df_)) + len_train: round(0.5*len(df_)) + len_train + len_val]


In [ ]:
to_df(x_val[0, 0])

In [ ]:
to_df(x_val[1, 0]).tail()

In [ ]:
to_df(y_val[0])

In [ ]:
x, y = list(val_iter)[0]
print(x.shape, y.shape)

In [ ]:
best_model.eval()
with torch.no_grad():
    x, y = list(val_iter)[0]
    y_pred = best_model(x).view(len(x), -1)
    plot(y, y_pred, 0)
        

In [ ]:
print(best_model(x).shape)
print(best_model(x).view(len(x), -1).shape)

In [ ]:
best_model(x).reshape(len(x), -1).shape

In [ ]:
len(x)

In [ ]:
i = 0
pd.DataFrame(scaler.inverse_transform(np.expand_dims(y[i].cpu().numpy(), axis=0)))

In [ ]:
data[(data["rt_plant_id"] == 0) & (data["forecast_dt"] > "2019-02-15") & (data["forecast_dt"] < "2019-02-16")]

In [ ]:
data[(data["rt_plant_id"] == 0) & (abs(data["production"] - 65.39) <= 0.02)]

In [ ]:
results = {}
for j in data.rt_plant_id.unique():
    m_ = data[(data["rt_plant_id"] == j)]["production"]
    results[j] = {}
    for i in range(len(m_) - 50):
        results[j][i] = np.sqrt(np.sum((m_[i: i + 50].values - n_) ** 2))

In [ ]:
a_ = pd.melt(pd.DataFrame.from_dict(results).reset_index(), id_vars="index")
a_.columns = ["time", "rt_plant_id", "error"]

In [ ]:
a_.sort_values("error")

In [ ]:
a_ = pd.DataFrame.from_records([results]).T.reset_index()
a_.columns = ["index", "error"]
a_.sort_values(by="error")

In [ ]:
n_ = scaler.inverse_transform(y_pred)[:, 0]

In [ ]:
plot(y, y_pred, i=51)

In [ ]:
y_pred.shape

In [ ]:
a = scaler.inverse_transform(y_pred.cpu().numpy())
a

In [ ]:
a.shape

In [ ]:
b = scaler.inverse_transform(y.cpu().numpy())
b

In [ ]:
b[:, 0]


In [ ]:
import matplotlib.pyplot as plt